In [1]:
import pandas as pd
import seoapikits

In [2]:
# https://cdn.dataforseo.com/v3/examples/python/python_Client.zip
keywords_list = pd.read_excel('./sample_data/await_finding_url.xlsx', engine='openpyxl')['company_name_cv'].to_list()[:5]

We read the sample data, we only pick the first 5, then Initialize the cls to search for

In [3]:
username = input("Enter the username: ")
password = input("Enter the password: ")
search_api_cls = seoapikits.SERP.SERPAPI(username=username, password=password)

After, we try to post the data, which means we send request to the seodata server, and we can get the task id, Then we could directly got the result by using get. However if request "get" so fast it could not finish search and cast error, so we can use timeout limit to make it to retry. default freq is 5 seconds and timeout is 0 seconds, However you can reset it to make better usage, here is the example

In [7]:
task_post_df = search_api_cls.tasks_seq_post(keyword_list=keywords_list,res_type='df')
task_post_df

,id,keyword
0,08251154-6469-0066-0000-0df17972155c,"""Johnnie"" Johnson Housing Trust Ltd."
1,08251154-6469-0066-0000-acee9082b3d4,051 Subs
2,08251154-6469-0066-0000-36bd33c0fd05,063 Loan Company Limited
3,08251154-6469-0066-0000-6509f7c175f9,0880 Santander Santibañez Sl
4,08251154-6469-0066-0000-1926aff8cbbf,0km Finance


In [8]:
task_get_df = search_api_cls.tasks_seq_get_df(task_post_df['id'], timeout_limit=0)
task_get_df

  0%|          | 0/5 [00:00<?, ?it/s]


ValueError: Error-timeout for not get a efficient response from Seodata

You can see, so fast search will result for not enough time for server to response. So we take time to retry.

In [9]:
task_post_df = search_api_cls.tasks_seq_post(keyword_list=keywords_list,res_type='df')
task_post_df

,id,keyword
0,08251154-6469-0066-0000-211f884541a5,"""Johnnie"" Johnson Housing Trust Ltd."
1,08251154-6469-0066-0000-bcbda1a0c597,051 Subs
2,08251154-6469-0066-0000-267a97f3fc19,063 Loan Company Limited
3,08251154-6469-0066-0000-f5e1e71f22a7,0880 Santander Santibañez Sl
4,08251154-6469-0066-0000-2247ce9d2cb0,0km Finance


In [10]:
task_get_df = search_api_cls.tasks_seq_get_df(task_post_df['id'], timeout_limit=30)

100%|██████████| 5/5 [00:11<00:00,  2.34s/it]


In [12]:
pd.merge(task_get_df, task_post_df, on='id', validate='many_to_one')

,type,rank_group,rank_absolute,domain,title,description,url,breadcrumb,id,keyword
0,organic,1,1,www.jjhousing.co.uk,Johnnie Johnson Housing: Housing Association,We are Johnnie Johnson Housing ... We are a no...,https://www.jjhousing.co.uk/,https://www.jjhousing.co.uk,08251154-6469-0066-0000-211f884541a5,"""Johnnie"" Johnson Housing Trust Ltd."
1,organic,2,2,www.findahomejjhousing.co.uk,Login - JJ Housing,Johnnie Johnson Housing Limited is a Registere...,https://www.findahomejjhousing.co.uk/login/,https://www.findahomejjhousing.co.uk › login,08251154-6469-0066-0000-211f884541a5,"""Johnnie"" Johnson Housing Trust Ltd."
2,organic,3,4,uk.linkedin.com,Johnnie Johnson Housing,Johnnie Johnson Housing is a not-for-profit ho...,https://uk.linkedin.com/company/johnniejohnson...,https://uk.linkedin.com › company › johnniejoh...,08251154-6469-0066-0000-211f884541a5,"""Johnnie"" Johnson Housing Trust Ltd."
3,organic,4,5,www.facebook.com,Johnnie Johnson Housing | Charity Organization,"Johnnie Johnson Housing, Cheadle Hulme. 1260 l...",https://www.facebook.com/jjhousingtrust/,https://www.facebook.com › jjhousingtrust,08251154-6469-0066-0000-211f884541a5,"""Johnnie"" Johnson Housing Trust Ltd."
4,organic,5,6,find-and-update.company-information.service.go...,"""johnnie"" johnson housing trust limited - Comp...","""JOHNNIE"" JOHNSON HOUSING TRUST LIMITED - Free...",https://find-and-update.company-information.se...,https://find-and-update.company-information.se...,08251154-6469-0066-0000-211f884541a5,"""Johnnie"" Johnson Housing Trust Ltd."
...,...,...,...,...,...,...,...,...,...,...
396,organic,95,96,www.autozel.com,Do you need Personal Finance :: AutoZel.com | ...,"Do you need Personal Finance - AED 600,000. Ne...",https://www.autozel.com/CarDetails.aspx?CarID=...,https://www.autozel.com › CarDetails › Title=d...,08251154-6469-0066-0000-2247ce9d2cb0,0km Finance
397,organic,96,97,www.autoexpress.co.uk,Top 10 cheapest cars to buy 2023,"... miles on the clock, but the appeal of a br...",https://www.autoexpress.co.uk/best-cars-vans/3...,https://www.autoexpress.co.uk › ... › Best car...,08251154-6469-0066-0000-2247ce9d2cb0,0km Finance
398,organic,97,98,www.rionegro.com.ar,La Justicia falló a favor de un hombre de Vied...,S.A. cambió el precio del auto 0km y el client...,https://www.rionegro.com.ar/justicia/la-justic...,https://www.rionegro.com.ar › justicia,08251154-6469-0066-0000-2247ce9d2cb0,0km Finance
399,organic,98,99,www.subaru.com,"Subaru Cars, SUVs, Crossovers & Hybrids | Suba...",Finance and lease offers cannot be combined wi...,https://www.subaru.com/index.html,https://www.subaru.com,08251154-6469-0066-0000-2247ce9d2cb0,0km Finance
